<a href="https://colab.research.google.com/github/nitron-alpha-kplr/Cloner-Netflix-avec-DataStax-DB-et-GraphQL/blob/main/Copie_de_1_Web_Scrapping_with_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutoriel : Extraire les articles d'une revue d'intellingence artificielle selon les catégories**

![image](https://user-images.githubusercontent.com/123748165/224306735-142be7ed-a969-458e-9bd3-65360d593a3e.png)

- Afin de pouvoir utiliser Selenium il faut d’abord installer le Web Driver associé au navigateur que l’on souhaite utiliser.


- Vous devez ensuite installer Selenium et le Web Driver dans votre environnement.

- Pour ce faire, on va utiliser dans notre notebook les commandes suivantes :

In [ ]:

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser
!pip install selenium-stealth
!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

 This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from Chrome into Chromium for you to use Netflix (or other websites using Encrypted Media Extensions) in Chromium.

1. Download and install Chrome (or extract the necessary files, if

On importe les modules dont on aura besoin :

In [ ]:

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from urllib.error import HTTPError
from time import time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium_stealth import stealth

Enfin, on modifie quelques options du Chrome Driver pour pouvoir l’utiliser depuis un notebook :

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
stealth(driver,
              languages=["fr-FR", "fr"],
              vendor="Google Inc.",
              platform="Win32",
              webgl_vendor="Intel Inc.",
              renderer="Intel Iris OpenGL Engine",
              fix_hairline=True,
          )

<ipython-input-3-3a4dc5b0e958>:5: DeprecationWarning: use options instead of chrome_options
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
<ipython-input-3-3a4dc5b0e958>:6: DeprecationWarning: use options instead of chrome_options
  driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)


# **Web Scraping des liens des articles**
Pour pouvoir faire correctement du Web Scraping, il faut se poser deux questions :

- Comment est construit le site web ?

- Quelles données souhaite-t-on recuperer ?

Pour pouvoir avoir une idee de l’architecture du site web que l’on etudie on peut utiliser une fonctionnalité tres utile disponible dans tous les navigateurs : Inspecter l’element.

![image](https://user-images.githubusercontent.com/123748165/224308453-e6a3ac7f-2d9e-4b14-a905-6bdff6fb1a2a.png)

In [ ]:
website="https://larevueia.fr/"

In [ ]:
driver.get("https://larevueia.fr/")

- Dans notre cas, on se rend vite compte que les articles sont repartis en catégories et que chaque categorie contient plusieurs pages. 

- On peut donc commencer par recuperer tout d’abord les liens des articles pour ensuite pouvoir recuperer les donnees article par article.

- Pour ce faire, on va tout d’abord creer une liste contenant l’ensemble des categories de notre site :

In [ ]:
categories = ['ethique', 'nlp', 'evenements', 'ml-dl', 'data-science', 'vision']

On peut donc facilement explorer le site en parcourant les liens des pages des differentes categories :

In [ ]:
articles_links = []
for category in categories:
  category_link = website + category
  driver.get(category_link)
  href_links = driver.find_elements(By.XPATH,'//h2/a[@href]')
  for href_link in href_links:
    articles_links.append([href_link.get_attribute('href'), category])

- Pour expliquer plus precisement ce que fait le code ci-dessus, on peut dire que l’on parcourt tout d’abord la liste des categories, pour chaque categorie on crée une variable, qui contient son URL.

- Puis on ouvre la page avec la fonction get() de notre driver.

- On utilise ensuite la fonction find_elements_by_xpath() pour rechercher les liens href se trouvant à l’interieur de balise h2 (ceci decoule directement de notre analyse du site avec la fonctionnalité Inspecter l’element de notre navigateur, vous pouvez copier le xpath directement sur le navigateur).

- Finalement, on ne recupere que le lien de l’article avec la fonction get_attribute(), et on ajoute donc à notre liste le lien de l’article ainsi que la categorie à laquelle il correspond (ce qui peut être utile plus tard pour faire du NLP avec un modele de classification par exemple).

- On pourrait aller plus loin et rajouter une deuxieme boucle pour incrementer à chaque fois le nombre de pages afin de parcourir tous les articles de chaque categorie, et pas seulement ceux presents sur la premiere page.

# **Web Scraping du contenu des articles avec Selenium**

- Une fois l’ensemble des liens d’articles recuperes, la prochaine etape est d’analyser le contenu de la page pour chaque article. 

- Encore une fois, on utilise la fonctionnalite Inspecter l’element, afin d’identifier la structure de la page et de retrouver les elements à scraper.

- On se rend vite compte que dans notre cas le contenu de l’article est stocké dans une classe nommee article-post. On va donc recuperer le contenu texte de cette classe et tout stocker dans un data frame avec Pandas :

In [ ]:
df_articles = pd.DataFrame()
for article_link in articles_links:
  driver.get(article_link[0])
  content_scrap = driver.find_elements(By.CLASS_NAME,'article-post')
  if (len(content_scrap) != 0):
    content = content_scrap[0].text
  article = {'category' : article_link[1], 'content' : content}
  df_article = pd.DataFrame(article, index = [0])
  df_articles = df_articles.append(df_article, ignore_index=True)

<ipython-input-10-a384f2cf141f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_articles = df_articles.append(df_article, ignore_index=True)
<ipython-input-10-a384f2cf141f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_articles = df_articles.append(df_article, ignore_index=True)
<ipython-input-10-a384f2cf141f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_articles = df_articles.append(df_article, ignore_index=True)
<ipython-input-10-a384f2cf141f>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_articles = df_articles.append(df_article, ignore_index=True)
<ipython-input-10-a384f2cf141f>:9: FutureWarning: The frame.appe

On se retrouve donc avec un data frame contenant nos articles, avec à chaque fois la categorie à laquelle il correspond. On peut finalement transformer ce data frame en fichier csv, afin de stocker les données :

In [ ]:
df_articles.to_csv('la_revue_ia_articles.csv')

In [ ]:
df_articles

,category,content
0,ethique,L’intelligence artificielle a conquis la plupa...
1,ethique,"Le web2 est cassé. Et comme lui, l’IA est cass..."
2,ethique,Un des avantages des réseaux de neurones résid...
3,ethique,Depuis qu’Alan Turing a introduit la question ...
4,ethique,"Dans cet article, on vous explique comment uti..."
5,ethique,"Il y a 3 ans, après la tech expo de San Franci..."
6,ethique,Si vous vous intéressez aux modèles de traitem...
7,ethique,Ces derniers temps je me suis mis à scaner tou...
8,ethique,Les applications de l’intelligence artificiell...
9,ethique,On met souvent en avant les prouesses des syst...
